In [ ]:
import pandas as pd

# Sampling rate
sampling_rate = 2000  # Hz

# === Function to load and process behavioral data ===
def load_behavioral(session_name, subject_id):
    # Load CSV
    filename = f"data/Behavioral_data/{session_name}Subject{subject_id}.csv"
    df = pd.read_csv(filename)
    
    # Filter out "Rest" blocks
    df = df[df['n_back'] != 'Rest']
    
    # Clean n_back
    df['n_back'] = df['n_back'].str.strip().str.lower()
    
    # Map to 1_back / 3_back
    def map_nback(text):
        if 'one back' in text:
            return '1_back'
        elif 'three back' in text:
            return '3_back'
        else:
            return None
    
    df['n_back_task'] = df['n_back'].apply(map_nback)
    
    # Correctness
    df['correct'] = df['Response'] == df['Correct_Response']
    
    # Group by TrialNumber (block) and n_back
    trial_summary = df.groupby(['TrialNumber', 'n_back_task']).agg(
        accuracy=('correct', 'mean'),
        mean_rt=('Response_Time', 'mean')
    ).reset_index()
    
    # Add session label
    trial_summary['session'] = session_name.lower()
    
    return trial_summary

# === Main loop over participants ===
participants = ['3F', '4F', '6M', '8M', '11F']
all_data = []

for pid in participants:
    subject_num = pid[:-1]
    gender = pid[-1]
    
    # Load behavioral summaries
    calming_behav = load_behavioral('Calming', subject_num)
    vexing_behav = load_behavioral('Vexing', subject_num)
    
    # Combine sessions — preserves session order (Calming first, then Vexing)
    behavior_df = pd.concat([calming_behav, vexing_behav], ignore_index=True)
    
    # Load EDA and TRIGGERS_BLOCK (block-level triggers!)
    eda_path = f"data/Biopac_data/EDA/Subject{subject_num}{gender}_EDA.csv"
    triggers_path = f"data/Biopac_data/Timing/Subject{subject_num}{gender}_Triggers_block.csv"
    
    eda_df = pd.read_csv(eda_path, header=None, names=['EDA'])
    triggers_df = pd.read_csv(triggers_path)
    
    print(f"\nParticipant {pid} → triggers_df shape: {triggers_df.shape} (should be ~8 rows, wide format)")

    # === Loop through behavioral rows — IN ORIGINAL ORDER ===
    for idx, behav_row in behavior_df.sort_values(['session', 'TrialNumber']).iterrows():
        session_label = behav_row['session']
        nback_label = behav_row['n_back_task']
        trial_number = behav_row['TrialNumber']
        
        found = False
        
        # Search for first unused matching trigger
        for t_idx, t_row in triggers_df.iterrows():
            start_col = f"{session_label}_{nback_label}_start"
            end_col = f"{session_label}_{nback_label}_end"
            
            if pd.notna(t_row[start_col]) and pd.notna(t_row[end_col]):
                # Process this EDA slice
                start_time = t_row[start_col]
                end_time = t_row[end_col]
                
                start_idx = int(start_time * sampling_rate)
                end_idx = int(end_time * sampling_rate)
                
                eda_slice = eda_df['EDA'].iloc[start_idx:end_idx]
                mean_eda = eda_slice.mean()
                
                # Mark this trigger as used
                triggers_df.at[t_idx, start_col] = None
                triggers_df.at[t_idx, end_col] = None
                
                # Append row
                all_data.append({
                    'participant': pid,
                    'trial': len(all_data) + 1,
                    'condition': f"{session_label.capitalize()} {nback_label.replace('_', '-').capitalize()}",
                    'mean_eda': 0 if pd.isna(mean_eda) else mean_eda,
                    'accuracy': 0 if pd.isna(behav_row['accuracy']) else behav_row['accuracy'],
                    'mean_rt': 0 if pd.isna(behav_row['mean_rt']) else behav_row['mean_rt']
                })
                
                found = True
                break
        
        if not found:
            print(f"WARNING: No matching trigger found for {session_label} {nback_label} TrialNumber {trial_number}")

    # Participant progress summary
    trials_processed = len(all_data) // len(participants)
    print(f"Participant {pid}: {trials_processed} trials processed (expected: 32)")

# Combine all participants into one DataFrame
combined_df = pd.DataFrame(all_data)

# Optional: save to CSV
combined_df.to_csv('df.csv', index=False)

# Preview result
print(f"\nFinal combined_df shape: {combined_df.shape}")
print(combined_df.head())


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [8]:
import pandas as pd

# Sampling rates
EDA_SAMPLING_RATE = 2000      # Hz
OHB_SAMPLING_RATE = 7.6294    # Hz

# === Function to load and process behavioral data ===
def load_behavioral(session_name, subject_id):
    filename = f"data/Behavioral_data/{session_name}Subject{subject_id}.csv"
    df = pd.read_csv(filename)
    df = df[df['n_back'] != 'Rest']
    df['n_back'] = df['n_back'].str.strip().str.lower()

    def map_nback(text):
        if 'one back' in text:
            return '1_back'
        elif 'three back' in text:
            return '3_back'
        return None
    df['n_back_task'] = df['n_back'].apply(map_nback)

    df['correct'] = df['Response'] == df['Correct_Response']
    summary = (
        df.groupby(['TrialNumber', 'n_back_task'])
          .agg(accuracy=('correct', 'mean'), mean_rt=('Response_Time', 'mean'))
          .reset_index()
    )
    summary['session'] = session_name.lower()
    return summary

# === Main processing ===
participants = ['3F', '4F', '6M', '8M', '11F']
all_data = []

for pid in participants:
    subj, gender = pid[:-1], pid[-1]
    calming = load_behavioral('Calming', subj)
    vexing = load_behavioral('Vexing', subj)
    behav_df = pd.concat([calming, vexing], ignore_index=True)

    # Load EDA data and triggers (first 8 cols)
    eda = pd.read_csv(
        f"data/Biopac_data/EDA/Subject{subj}{gender}_EDA.csv",
        header=None, names=['EDA']
    )
    trig_eda = pd.read_csv(
        f"data/Biopac_data/Timing/Subject{subj}{gender}_Triggers_block.csv"
    ).iloc[:, :8]

    # Load OHb raw data as a single series and its triggers (first 8 cols)
    raw_ohb = pd.read_csv(
        f"data/fNIRS_data/ohb/Subject{subj}{gender}_ohb.csv",
        header=None
    )
    # Flatten to 1D and create a Series
    ohb = pd.Series(raw_ohb.values.flatten(), name='ohb')
    trig_ohb = pd.read_csv(
        f"data/fNIRS_data/Subject{subj}{gender}_Triggers.csv"
    ).iloc[:, :8]

    print(f"\nParticipant {pid}: EDA triggers {trig_eda.shape}, OHb triggers {trig_ohb.shape}")

    # Iterate through trials in original order
    for _, behav in behav_df.sort_values(['session', 'TrialNumber']).iterrows():
        sess = behav['session']
        task = behav['n_back_task']
        start_col = f"{sess}_{task}_start"
        end_col   = f"{sess}_{task}_end"
        found = False

        # Check each trigger row index
        for idx in trig_eda.index:
            e_start = trig_eda.at[idx, start_col]
            e_end   = trig_eda.at[idx, end_col]
            if pd.notna(e_start) and pd.notna(e_end):
                # EDA slice and mean
                s_e = int(e_start * EDA_SAMPLING_RATE)
                e_e = int(e_end   * EDA_SAMPLING_RATE)
                mean_eda = eda['EDA'].iloc[s_e:e_e].mean()
                trig_eda.at[idx, start_col] = None
                trig_eda.at[idx, end_col]   = None

                # OHb slice and mean using its own triggers
                o_start = trig_ohb.at[idx, start_col]
                o_end   = trig_ohb.at[idx, end_col]
                s_o = int(o_start * OHB_SAMPLING_RATE)
                e_o = int(o_end   * OHB_SAMPLING_RATE)
                mean_ohb = ohb.iloc[s_o:e_o].mean()
                trig_ohb.at[idx, start_col] = None
                trig_ohb.at[idx, end_col]   = None

                all_data.append({
                    'participant': pid,
                    'condition': f"{sess.capitalize()} {task.replace('_','-').capitalize()}",
                    'mean_eda': mean_eda,
                    'mean_ohb': 0 if pd.isna(mean_ohb) else mean_ohb,
                    'accuracy': behav['accuracy'],
                    'mean_rt': behav['mean_rt']
                })
                found = True
                break

        if not found:
            print(f"WARNING: No trigger for {sess} {task} trial {behav['TrialNumber']}")

    processed = len(all_data) // len(participants)
    print(f"Participant {pid} processed {processed} trials (expected 32)")

# Save with full precision
df_combined = pd.DataFrame(all_data)
df_combined.to_csv('data/df.csv', index=False, float_format='%.18e')

print(f"\nFinal combined shape: {df_combined.shape}")
print(df_combined.head())





Participant 3F: EDA triggers (8, 8), OHb triggers (8, 8)
Participant 3F processed 6 trials (expected 32)

Participant 4F: EDA triggers (8, 8), OHb triggers (8, 8)
Participant 4F processed 12 trials (expected 32)

Participant 6M: EDA triggers (8, 8), OHb triggers (8, 8)
Participant 6M processed 19 trials (expected 32)

Participant 8M: EDA triggers (8, 8), OHb triggers (8, 8)
Participant 8M processed 25 trials (expected 32)

Participant 11F: EDA triggers (8, 8), OHb triggers (8, 8)
Participant 11F processed 32 trials (expected 32)

Final combined shape: (160, 6)
  participant       condition  mean_eda  mean_ohb  accuracy     mean_rt
0          3F  Calming 1-back  7.537035  0.008164  0.863636  564.409091
1          3F  Calming 3-back  7.053329  0.003207  0.681818  488.136364
2          3F  Calming 3-back  6.492472 -0.000443  0.727273  492.272727
3          3F  Calming 1-back  6.135949 -0.000339  0.954545  308.954545
4          3F  Calming 3-back  5.944514  0.000300  0.681818  656.863636
